# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846870891814                   -0.70    4.5
  2   -7.852319684177       -2.26       -1.53    1.0
  3   -7.852645982059       -3.49       -2.53    3.0
  4   -7.852646677193       -6.16       -3.36    2.8
  5   -7.852646685861       -8.06       -4.77    1.0
  6   -7.852646686724       -9.06       -5.20    4.5
  7   -7.852646686730      -11.27       -6.00    2.0
  8   -7.852646686730      -12.73       -7.55    1.5
  9   -7.852646686730      -14.75       -7.91    2.5
 10   -7.852646686730   +    -Inf       -8.64    1.0
 11   -7.852646686730      -15.05       -9.37    2.5
 12   -7.852646686730      -15.05      -10.09    2.8
 13   -7.852646686730      -15.05      -10.08    1.0
 14   -7.852646686730   +  -15.05       -9.38    2.0
 15   -7.852646686730   +    -Inf      -10.14    2.0
 16   -7.852646686730   +  -15.05       -9.74    1.0
 17   -7.852646686730      -15.05      -10.93 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846858235620                   -0.70           5.0
  2   -7.852528548614       -2.25       -1.64   0.80    2.2
  3   -7.852634095623       -3.98       -2.76   0.80    1.0
  4   -7.852646444247       -4.91       -3.20   0.80    2.0
  5   -7.852646666454       -6.65       -4.00   0.80    1.5
  6   -7.852646686227       -7.70       -4.62   0.80    1.8
  7   -7.852646686716       -9.31       -5.51   0.80    1.5
  8   -7.852646686730      -10.87       -6.79   0.80    2.2
  9   -7.852646686730      -12.79       -7.04   0.80    2.5
 10   -7.852646686730      -14.27       -7.59   0.80    1.0
 11   -7.852646686730   +    -Inf       -8.57   0.80    1.0
 12   -7.852646686730   +    -Inf       -9.31   0.80    2.2
 13   -7.852646686730      -15.05      -10.13   0.80    1.5
 14   -7.852646686730      -15.05      -10.41   0.80    2.2
 15   -7.852646686730   +  -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.413140e+01     3.201596e+00
 * time: 0.42616987228393555
     1     1.138610e+00     1.588333e+00
 * time: 0.6279528141021729
     2    -1.479028e+00     1.906306e+00
 * time: 0.7418577671051025
     3    -3.485025e+00     1.698690e+00
 * time: 0.775446891784668
     4    -4.627127e+00     1.515257e+00
 * time: 0.8093068599700928
     5    -6.454648e+00     9.990811e-01
 * time: 0.8428688049316406
     6    -7.302145e+00     5.451949e-01
 * time: 0.8764429092407227
     7    -7.582108e+00     5.443144e-01
 * time: 0.8999028205871582
     8    -7.770723e+00     2.150627e-01
 * time: 0.9235029220581055
     9    -7.819831e+00     7.613228e-02
 * time: 0.9469759464263916
    10    -7.837884e+00     8.268728e-02
 * time: 0.970477819442749
    11    -7.842544e+00     5.805534e-02
 * time: 0.9938058853149414
    12    -7.847603e+00     4.305388e-02
 * time: 1.0173068046569824
    13    -7.850851e+00     2.491178e-02
 * time: 1.04096984863

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846844377738                   -0.70    4.8
  2   -7.852318283431       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686711                   -2.54
  2   -7.852646686730      -10.73       -5.99
  3   -7.852646686730   +  -15.05      -12.64


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.0564373852783712e-13
|ρ_newton - ρ_scfv| = 3.723124034942094e-13
|ρ_newton - ρ_dm|   = 6.350542655076842e-10
